In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

# 2020년 사용자 개인 정보 가져오기 및 확인

In [2]:
info_df = pd.read_csv('./2020 data/user_info_2020.csv')
info_df=info_df.sort_values(['userId'],axis=0).reset_index(drop=True)
info_df.loc[info_df['gender']=='M','gender']=0
info_df.loc[info_df['gender']=='F','gender']=1
info_df.loc[info_df['handed']=='Left','handed']=0
info_df.loc[info_df['handed']=='Right','handed']=1
info_df

,userId,gender,age,handed,height,weight,startDt,endDt
0,user01,0,27,0,167,76,2020-08-30,2020-09-27
1,user02,1,25,0,168,81,2020-08-31,2020-09-27
2,user03,1,32,1,166,62,2020-08-29,2020-09-27
3,user04,0,28,1,171,70,2020-08-31,2020-09-27
4,user05,0,31,1,177,75,2020-08-31,2020-09-27
5,user06,1,22,1,164,70,2020-09-11,2020-10-08
6,user07,0,30,1,173,83,2020-08-31,2020-09-27
7,user08,0,32,1,180,84,2020-08-31,2020-09-25
8,user09,0,20,1,170,80,2020-08-31,2020-09-27
9,user10,0,25,1,184,77,2020-08-30,2020-09-27


In [3]:
info_df.to_csv('./2020 data/update_info_2020.csv',index=False)

# 2020년 잠 센서 데이터 처리

In [4]:
sleep_df = pd.read_csv('./2020 data/user_sleep_2020.csv')
sleep_df = sleep_df.drop(['timezone'],axis=1)
# 한국 시간 기준으로 설정하기 위해 timedelta(hours=9)를 진행함
sleep_df['startDt'] = pd.to_datetime(sleep_df['startDt'],unit='s') + timedelta(hours=9)
sleep_df['endDt'] = pd.to_datetime(sleep_df['endDt'],unit='s') + timedelta(hours=9)
sleep_df['lastUpdate'] = pd.to_datetime(sleep_df['lastUpdate'],unit='s') + timedelta(hours=9)
sleep_df.drop('lastUpdate',axis=1,inplace=True)

tmp = pd.to_datetime(sleep_df['date']) - timedelta(days=1)
tmp = tmp.apply(lambda x: x.strftime('%Y-%m-%d'))
sleep_df['date'] = tmp

#여기까지가 당일 date가 startDt, endDt에 맞춰줌
#이제는 당일 전 데이터를 당일 데이터에 붙여줘야함
tmp = pd.DataFrame(columns=["Pre_"+sleep_df.keys()[2:]])
a = pd.to_datetime(sleep_df['date']) -  timedelta(days=1)
a = a.apply(lambda x: x.strftime('%Y-%m-%d'))
tmp['Pre_date']= a

# 이전 데이트의 값을 tmp에 누적시키는 과정
pre_date = 0
for i in range(1,len(sleep_df)):
    if ((sleep_df['date'][i-1]) == (tmp['Pre_date'].iloc[i])).bool() or ((tmp['Pre_date'].iloc[i]) == pre_date).bool():
        tmp.iloc[i,:-1] = sleep_df.iloc[i-1,2:]
    pre_date = tmp['Pre_date'].iloc[i]

sleep_df=pd.concat([sleep_df,tmp],axis=1)
sleep_df

,userId,date,startDt,endDt,wakeupduration,lightsleepduration,deepsleepduration,wakeupcount,durationtosleep,remsleepduration,...,"(Pre_hr_min,)","(Pre_hr_max,)","(Pre_rr_average,)","(Pre_rr_min,)","(Pre_rr_max,)","(Pre_breathing_disturbances_intensity,)","(Pre_snoring,)","(Pre_snoringepisodecount,)","(Pre_sleep_score,)","(Pre_date,)"
0,user24,2020-08-29,2020-08-29 22:46:00,2020-08-30 07:41:00,3840,13800,9000,3,1740,5280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-28
1,user24,2020-08-30,2020-08-31 00:09:00,2020-08-31 07:19:00,4740,9540,6360,1,3180,4980,...,63,82,15,10,20,10,840,1,86,2020-08-29
2,user24,2020-08-31,2020-08-31 23:59:00,2020-09-01 06:20:00,1920,8760,4740,1,1440,7440,...,60,82,15,11,20,8,2700,3,52,2020-08-30
3,user24,2020-09-01,2020-09-02 01:57:00,2020-09-02 07:33:00,2760,7740,7440,1,1380,2220,...,58,94,15,12,23,8,0,0,61,2020-08-31
4,user24,2020-09-02,2020-09-03 00:33:00,2020-09-03 07:33:00,3900,11640,7620,1,1680,2040,...,60,80,15,12,19,-1,240,1,40,2020-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,user06,2020-10-04,2020-10-05 02:04:00,2020-10-05 10:37:00,8460,11820,4560,1,7740,5940,...,42,70,17,10,25,14,2820,4,75,2020-10-03
611,user06,2020-10-05,2020-10-06 03:46:00,2020-10-06 11:27:00,1920,13200,7500,1,1320,4680,...,43,81,17,12,22,13,2400,5,60,2020-10-04
612,user06,2020-10-06,2020-10-07 01:22:00,2020-10-07 07:36:00,12420,8520,1500,3,10440,0,...,47,84,17,10,23,10,1380,4,79,2020-10-05
613,user06,2020-10-07,2020-10-08 02:17:00,2020-10-08 07:38:00,4380,9240,4560,2,2820,1080,...,42,76,16,12,20,-1,1620,4,20,2020-10-06


In [5]:
sleep_df.to_csv('./2020 data/update_user_sleep_2020.csv',index=False)

# 2020년 잠 설문조사 데이터 처리

In [6]:
survey_df = pd.read_csv('./2020 data/user_survey_2020.csv')
survey_df = survey_df.backfill(limit=1)
survey_df=survey_df.drop(survey_df[survey_df['amPm']=='pm'].index).reset_index(drop=True).drop(['amPm'],axis=1)

# 카페인과 음주 여부만 확인
survey_df.loc[survey_df['caffeine'].isnull() == False,'caffeine'] = 1
survey_df['caffeine'].fillna(0,inplace=True)
survey_df.loc[survey_df['alcohol'].isnull() == False,'alcohol'] = 1
survey_df['alcohol'].fillna(0,inplace=True)
survey_df=survey_df.drop(['cAmount(ml)','aAmount(ml)','startInput','endInput'],axis=1)

# 당일 이후 데이터를 당일 데이터에 붙히는 과정
tmp = pd.DataFrame(columns=["Post_"+survey_df.keys()[2:7]])
a = pd.to_datetime(survey_df['date']) +  timedelta(days=1)
a = a.apply(lambda x: x.strftime('%Y-%m-%d'))
tmp['Post_date']= a

post_date = 0
for i in range(len(survey_df)-1):
    if ((survey_df['date'][i+1]) == (tmp['Post_date'].iloc[i])).bool() or ((tmp['Post_date'].iloc[i]) == post_date).bool():
        tmp.iloc[i,:-1] = survey_df.iloc[i+1,2:7]
    post_date = tmp['Post_date'].iloc[i]

survey_df=pd.concat([survey_df,tmp],axis=1)
survey_df

,userId,date,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,pmStress,pmFatigue,caffeine,alcohol,"(Post_sleep,)","(Post_sleepProblem,)","(Post_dream,)","(Post_amCondition,)","(Post_amEmotion,)","(Post_date,)"
0,user24,2020-08-29,4.0,0.0,3.0,3.0,4.0,4.0,4.0,2.0,0,0,3.0,2.0,2.0,2.0,3.0,2020-08-30
1,user24,2020-08-30,3.0,2.0,2.0,2.0,3.0,4.0,5.0,2.0,1,0,3.0,2.0,2.0,3.0,3.0,2020-08-31
2,user24,2020-08-31,3.0,2.0,2.0,3.0,3.0,4.0,2.0,4.0,1,0,4.0,0.0,4.0,4.0,3.0,2020-09-01
3,user24,2020-09-01,4.0,0.0,4.0,4.0,3.0,5.0,4.0,3.0,1,0,4.0,0.0,4.0,2.0,4.0,2020-09-02
4,user24,2020-09-02,4.0,0.0,4.0,2.0,4.0,4.0,4.0,2.0,0,0,4.0,0.0,2.0,2.0,3.0,2020-09-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,user06,2020-10-04,4.0,0.0,4.0,4.0,4.0,3.0,2.0,3.0,0,0,4.0,0.0,4.0,4.0,3.0,2020-10-05
611,user06,2020-10-05,4.0,0.0,4.0,4.0,3.0,4.0,4.0,4.0,0,0,4.0,0.0,4.0,4.0,4.0,2020-10-06
612,user06,2020-10-06,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,0,0,2.0,9.0,4.0,2.0,3.0,2020-10-07
613,user06,2020-10-07,2.0,9.0,4.0,2.0,3.0,5.0,2.0,3.0,1,0,3.0,0.0,4.0,2.0,2.0,2020-10-08


In [7]:
survey_df.to_csv('./2020 data/update_user_survey_2020.csv',index=False)

# 설문조사와 잠 데이터 userId, date 기준으로 병합

In [8]:
survey_sleep_df = survey_df.merge(sleep_df, how = 'outer', on=['userId','date'])
survey_sleep_df

,userId,date,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,pmStress,pmFatigue,...,"(Pre_hr_min,)","(Pre_hr_max,)","(Pre_rr_average,)","(Pre_rr_min,)","(Pre_rr_max,)","(Pre_breathing_disturbances_intensity,)","(Pre_snoring,)","(Pre_snoringepisodecount,)","(Pre_sleep_score,)","(Pre_date,)"
0,user24,2020-08-29,4.0,0.0,3.0,3.0,4.0,4.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-28
1,user24,2020-08-30,3.0,2.0,2.0,2.0,3.0,4.0,5.0,2.0,...,63,82,15,10,20,10,840,1,86,2020-08-29
2,user24,2020-08-31,3.0,2.0,2.0,3.0,3.0,4.0,2.0,4.0,...,60,82,15,11,20,8,2700,3,52,2020-08-30
3,user24,2020-09-01,4.0,0.0,4.0,4.0,3.0,5.0,4.0,3.0,...,58,94,15,12,23,8,0,0,61,2020-08-31
4,user24,2020-09-02,4.0,0.0,4.0,2.0,4.0,4.0,4.0,2.0,...,60,80,15,12,19,-1,240,1,40,2020-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,user27,2020-10-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48,74,13,9,22,4,0,0,81,2020-10-02
703,user27,2020-10-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51,71,14,10,24,-1,0,0,28,2020-10-03
704,user27,2020-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53,66,13,9,18,-1,0,0,20,2020-10-04
705,user27,2020-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52,74,14,10,25,10,0,0,65,2020-10-05


# 병합된 데이터 크기 확인

In [9]:
survey_sleep_df['userId'].value_counts()

user01    48
user27    42
user12    36
user04    35
user07    32
user10    32
user26    32
user24    31
user23    31
user25    31
user21    31
user03    31
user06    31
user02    30
user30    30
user08    30
user05    30
user11    29
user22    29
user29    29
user09    29
user28    28
Name: userId, dtype: int64

# csv 파일로 내보내기

In [10]:
survey_sleep_df.to_csv('./2020 data/2020_survey_sleep.csv',index=False)

# 왜 sleep_df와 survey_df가 다른지 확인 과정

In [11]:
sleep_df[sleep_df['userId'] == 'user29']

,userId,date,startDt,endDt,lastUpdate,wakeupduration,lightsleepduration,deepsleepduration,wakeupcount,durationtosleep,...,hr_average,hr_min,hr_max,rr_average,rr_min,rr_max,breathing_disturbances_intensity,snoring,snoringepisodecount,sleep_score
420,user29,2020-08-30,2020-08-31 01:26:00,2020-08-31 09:11:00,2020-08-31 11:14:11,6720,11400,4020,3,1980,...,58,53,75,16,10,25,12,240,1,21
421,user29,2020-08-31,2020-09-01 00:46:00,2020-09-01 09:09:00,2020-09-01 11:12:10,2160,11820,7200,0,1080,...,59,53,73,16,12,25,12,600,3,90
422,user29,2020-09-01,2020-09-02 00:07:00,2020-09-02 09:32:00,2020-09-02 11:35:08,2400,13440,6300,1,1980,...,58,53,77,16,8,26,12,0,0,96
423,user29,2020-09-02,2020-09-03 00:34:00,2020-09-03 09:48:00,2020-09-03 11:51:09,3000,16080,6660,1,2280,...,56,50,73,16,10,27,10,0,0,89
424,user29,2020-09-03,2020-09-04 01:26:00,2020-09-04 09:26:00,2020-09-04 11:29:09,2040,12300,8700,1,1680,...,54,49,66,16,12,25,10,0,0,91
425,user29,2020-09-09,2020-09-09 23:58:00,2020-09-10 09:55:00,2020-09-11 03:49:37,7800,11460,8340,3,5040,...,56,50,76,16,12,27,9,0,0,80
426,user29,2020-09-10,2020-09-10 23:08:00,2020-09-11 09:01:00,2020-09-11 11:05:02,4200,16860,5760,4,1260,...,55,45,71,17,11,25,23,0,0,81
427,user29,2020-09-11,2020-09-12 01:48:00,2020-09-12 11:13:00,2020-09-12 11:50:09,2220,14820,9960,1,2220,...,64,55,84,17,12,26,12,0,0,96
428,user29,2020-09-12,2020-09-12 22:43:00,2020-09-13 12:08:00,2020-09-13 14:13:12,7440,19440,10200,5,1260,...,55,44,69,17,11,25,20,0,0,89
429,user29,2020-09-13,2020-09-14 00:36:00,2020-09-14 09:10:00,2020-09-14 11:13:10,8400,11700,5520,3,7320,...,55,47,65,15,12,25,12,0,0,62


In [12]:
survey_df[survey_df['userId'] == 'user29']

,userId,date,startInput,endInput,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,pmStress,pmFatigue,caffeine,cAmount(ml),alcohol,aAmount(ml)
422,user29,2020-08-30,2020-08-31 00:27,2020-08-31 00:28,4.0,0.0,4.0,4.0,4.0,2.0,2.0,2.0,coffee,300.0,NaN,NaN
423,user29,2020-08-31,2020-08-31 09:08,2020-08-31 09:09,3.0,0.0,4.0,2.0,3.0,3.0,2.0,3.0,NaN,NaN,NaN,NaN
424,user29,2020-09-01,2020-09-01 08:51,2020-09-01 08:51,4.0,0.0,4.0,2.0,3.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN
425,user29,2020-09-02,2020-09-02 09:31,2020-09-02 09:31,4.0,0.0,4.0,2.0,3.0,3.0,3.0,2.0,NaN,NaN,NaN,NaN
426,user29,2020-09-03,2020-09-03 08:41,2020-09-03 08:41,4.0,0.0,4.0,2.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN
427,user29,2020-09-04,2020-09-04 09:21,2020-09-04 09:21,4.0,0.0,4.0,2.0,3.0,4.0,3.0,3.0,NaN,NaN,NaN,NaN
428,user29,2020-09-05,2020-09-05 22:17,2020-09-05 22:19,2.0,2.0,4.0,2.0,3.0,4.0,3.0,4.0,coffee,300.0,NaN,NaN
429,user29,2020-09-06,2020-09-06 10:48,2020-09-06 10:48,4.0,0.0,4.0,4.0,4.0,3.0,4.0,2.0,coffee,300.0,NaN,NaN
430,user29,2020-09-07,2020-09-07 09:40,2020-09-07 09:41,3.0,5.0,4.0,2.0,3.0,3.0,3.0,2.0,NaN,NaN,NaN,NaN
431,user29,2020-09-08,2020-09-08 09:54,2020-09-08 09:54,3.0,0.0,4.0,4.0,3.0,3.0,3.0,2.0,NaN,NaN,NaN,NaN


# 확인 결과: user29에 대한 예시- sleep_df에 8/30 ~ 9/14일 까지의 잠 센서 데이터가 들어 있지만, survey_df에 8/30 ~ 9/27일 까지의 데이터가 들어있음